In [10]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from pymongo import MongoClient
client = MongoClient("localhost", 27017, maxPoolSize=50)
db = client.book
collection = db['FullTimeMatches']
cursor = collection.find()
df =  pd.DataFrame(list(cursor))
pd.set_option('display.max_rows', 5000)
pd.options.display.max_columns = None
df_Eredevisie =  df[df.league_name == 'Eredivisie'  ]

df_minimal_Eredevisie =  df_Eredevisie[[ 
                                        'homeredUpTo75'
                                        ,'awayredUpTo75'
                                        ,'homeyellowUpTo75'
                                        ,'awayyellowUpTo75'
                                        ,'homeGoalsUpTo75'
                                        ,'awayGoalsUpTo75' 
                                        ,'resultAt60'
                                        ,'resultAt75'
                                        ,'goalScoredLast15'
                                           ,'HomeTeamWhileHomeAverageAttack'
                                        ,'AwayTeamWhileAwayAverageAttack'
                                        ,'AwayTeamWhileAwayAverageDefence'
                                        ,'HomeTeamWhileHomeAverageDefence'
                                      ]].copy()

df_minimal_Eredevisie['goalScoredLast15'] = df_minimal_Eredevisie['goalScoredLast15'] == 1

 
dummyresult = pd.get_dummies(df_minimal_Eredevisie['resultAt60'],prefix='resultAt60')


df_minimal_Eredevisie = pd.concat([df_minimal_Eredevisie,dummyresult], axis=1)


dummyresult = pd.get_dummies(df_minimal_Eredevisie['resultAt75'],prefix='resultAt75')
dummyresult.head()
df_minimal_Eredevisie = pd.concat([df_minimal_Eredevisie,dummyresult], axis=1)

df_minimal_Eredevisie['homeredUpTo75'] = df_minimal_Eredevisie['homeredUpTo75'].astype(np.int64)
df_minimal_Eredevisie['awayredUpTo75'] =df_minimal_Eredevisie['awayredUpTo75'].astype(np.int64)
df_minimal_Eredevisie['homeyellowUpTo75'] = df_minimal_Eredevisie['homeyellowUpTo75'].astype(np.int64)
df_minimal_Eredevisie['awayyellowUpTo75'] = df_minimal_Eredevisie['awayyellowUpTo75'].astype(np.int64)
df_minimal_Eredevisie['homeGoalsUpTo75'] = df_minimal_Eredevisie['homeGoalsUpTo75'].astype(np.int64)
df_minimal_Eredevisie['awayGoalsUpTo75'] = df_minimal_Eredevisie['awayGoalsUpTo75'].astype(np.int64)



feature_cols = [                             'homeredUpTo75'
                                        ,'awayredUpTo75'
                                        ,'homeyellowUpTo75'
                                        ,'awayyellowUpTo75'
                                        ,'homeGoalsUpTo75'
                                        ,'awayGoalsUpTo75' 
                                        ,'resultAt60_DRAW'
                                        ,'resultAt60_LOOSE'
                                        ,'resultAt60_WIN'
                                        ,'resultAt75_DRAW'
                                        ,'resultAt75_LOOSE'
                                        ,'resultAt75_WIN'
                                        ,'HomeTeamWhileHomeAverageAttack'
                                        ,'AwayTeamWhileAwayAverageAttack'
                                        ,'AwayTeamWhileAwayAverageDefence'
                                        ,'HomeTeamWhileHomeAverageDefence'
                
                                
                    
               ]


y =df_minimal_Eredevisie['goalScoredLast15']
X=df_minimal_Eredevisie[feature_cols]


from sklearn.model_selection  import train_test_split 

X_train, X_test, y_train , y_test = train_test_split(X,y,test_size=0.3, random_state=1)
from sklearn.linear_model import LogisticRegression 
logmodel = LogisticRegression() 
logmodel.fit(X_train,y_train)







/home/darth/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [11]:
predictions = logmodel.predict(X_test)

In [12]:
from sklearn.metrics import classification_report 

classification_report(y_test,predictions)

'              precision    recall  f1-score   support\n\n       False       0.52      0.59      0.55       400\n        True       0.51      0.43      0.47       387\n\n   micro avg       0.51      0.51      0.51       787\n   macro avg       0.51      0.51      0.51       787\nweighted avg       0.51      0.51      0.51       787\n'

In [13]:
from sklearn.metrics import confusion_matrix 

confusion_matrix(y_test,predictions)

array([[238, 162],
       [220, 167]])